In [ ]:
# === 1. KÖRNYEZET BEÁLLÍTÁSA ===
# Könyvtárak telepítése és importálása
!pip install -U torch sentence-transformers accelerate pyarrow pandas tqdm transformers

import pandas as pd
import numpy as np
import gc
import json
import pyarrow as pa
import pyarrow.parquet as pq
from sentence_transformers import SentenceTransformer
import torch
import time
from tqdm.auto import tqdm
from typing import List
from pathlib import Path
import os

# !!! KRITIKUS JAVÍTÁS: PyTorch memória töredezettségének kezelése !!!
# A hibaüzenet javaslata alapján beállítjuk ezt a környezeti változót,
# hogy a PyTorch rugalmasabban kezelje a GPU memóriát.
# Ezt minden más PyTorch művelet előtt kell megtenni.
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# GPU optimalizáció A100-hoz
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True

print(f"CUDA elérhető: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# === 2. KONFIGURÁCIÓ ===
# RunPod felhő környezethez igazított konfiguráció.

from pathlib import Path
import logging
import csv
import sys

# --- CSV OLVASÁSI LIMIT NÖVELÉSE ---
try:
    max_int = sys.maxsize
    while True:
        try:
            csv.field_size_limit(max_int)
            break
        except OverflowError:
            max_int = int(max_int / 10)
except (ValueError, TypeError):
    csv.field_size_limit(1_000_000_000)

INPUT_CSV_PATH = Path("/workspace/cleaned_data_for_embedding.csv")
OUTPUT_PARQUET_PATH = Path("/workspace/documents_with_embeddings.parquet")

MODEL_NAME = "Qwen/Qwen3-Embedding-0.6B"
EMBEDDING_DIMENSION = 1024
# !!! VÉGSŐ BIZTONSÁGI INTÉZKEDÉS: BATCH MÉRET CSÖKKENTÉSE !!!
BATCH_SIZE = 128  # Tovább csökkentjük 256-ról 128-ra

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

current_limit = csv.field_size_limit()
logger.info(f"CSV field size limit beállítva: {current_limit:,}")

logger.info(f"Input: {INPUT_CSV_PATH}")
logger.info(f"Output: {OUTPUT_PARQUET_PATH}")
logger.info(f"Modell: {MODEL_NAME}")
logger.info(f"Batch méret: {BATCH_SIZE}")

In [ ]:
# === 3. EMBEDDING GENERÁTOR OSZTÁLY ===
# Ez az osztály tiszta és önálló, csak az embedding generálásra fókuszál.
class EmbeddingGenerator:
    def __init__(self, model_name: str, batch_size: int, dimension: int, device: str = 'cuda'):
        self.model_name = model_name
        self.batch_size = batch_size
        self.dimension = dimension
        self.device = device if torch.cuda.is_available() else 'cpu'
        self.model = None
        logger.info(f"Generátor inicializálva a(z) '{self.device}' eszközön.")

    def load_model(self):
        if self.model is not None:
            logger.info("Modell már be van töltve.")
            return
        try:
            logger.info(f"'{self.model_name}' modell betöltése...")
            self.model = SentenceTransformer(self.model_name, device=self.device, trust_remote_code=True)
            self._warmup_model()
            logger.info("Modell sikeresen betöltve és bemelegítve.")
        except Exception as e:
            logger.error(f"Modell betöltési hiba: {e}")
            raise

    def _warmup_model(self):
        logger.info("Modell bemelegítése...")
        self.generate_embeddings(["melegítés"])
        self._cleanup_memory()
        logger.info("Bemelegítés kész.")

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        if self.model is None:
            raise RuntimeError("A modell nincs betöltve. Hívd meg a load_model() metódust.")
        try:
            embeddings = self.model.encode(
                texts, 
                batch_size=self.batch_size, 
                normalize_embeddings=True, 
                show_progress_bar=True, # Legyen progress bar a konzolon
                convert_to_numpy=True
            )
            if embeddings.shape[1] != self.dimension: # Biztonsági ellenőrzés
                logger.warning(f"Váratlan embedding dimenzió: {embeddings.shape[1]}. Korrekció {self.dimension}-ra.")
                embeddings = embeddings[:, :self.dimension]
            return embeddings.astype(np.float32)
        except Exception as e:
            # Részletesebb logolás a hiba jobb megértéséhez
            problematic_text_snippet = texts[0][:200] if texts else "Üres a szöveg lista"
            logger.error(f"!!! KRITIKUS HIBA AZ EMBEDDING GENERÁLÁSKOR !!!")
            logger.error(f"Hibaüzenet: {e}")
            logger.error(f"A hibát okozó batch első szövegének részlete (első 200 karakter): '{problematic_text_snippet}'")
            
            # Újra feldobjuk a hibát a teljes hiba-visszakövetésért (traceback)
            raise

    def _cleanup_memory(self):
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

In [ ]:
# === 4. FŐ FELDOLGOZÁSI FOLYAMAT ===
def create_metadata_json(row: pd.Series) -> str:
    metadata_cols = [col for col in row.index if col not in ['text', 'embedding']]
    metadata_dict = row[metadata_cols].dropna().to_dict()
    return json.dumps({k: str(v) for k, v in metadata_dict.items()}, ensure_ascii=False)

def main():
    logger.info("Feldolgozás indítása...")
    start_time = time.time()

    # Bemeneti adatok beolvasása
    if not INPUT_CSV_PATH.exists():
        error_msg = f"Hiba: A bemeneti fájl nem található: {INPUT_CSV_PATH}"
        logger.error(error_msg)
        raise FileNotFoundError(error_msg)
    
    logger.info(f"Bemeneti CSV beolvasása: {INPUT_CSV_PATH}")
    df = pd.read_csv(INPUT_CSV_PATH, engine='python', quoting=csv.QUOTE_ALL, on_bad_lines='warn')
    logger.info(f"{len(df):,} sor sikeresen beolvasva.")

    # Szövegek kinyerése és tisztítása
    df['text'] = df['text'].fillna('')
    texts_to_process = df['text'].astype(str).tolist()
    
    if not texts_to_process:
        logger.warning("Nincs feldolgozandó szöveg a bemeneti fájlban.")
        return

    # Embedding generátor inicializálása
    generator = EmbeddingGenerator(MODEL_NAME, BATCH_SIZE, EMBEDDING_DIMENSION)
    generator.load_model()

    # --- MEMÓRIAHATÉKONY FELDOLGOZÁS DARABOKBAN (CHUNK-OKBAN) ---
    logger.info("Embedding generálás megkezdése memóriahatékony, darabolt módszerrel.")
    all_embeddings = []
    # Biztonsági okokból csökkentett darabméret
    processing_chunk_size = 4096 

    for i in tqdm(range(0, len(texts_to_process), processing_chunk_size), desc="Adatdarabok feldolgozása"):
        batch_texts = texts_to_process[i:i + processing_chunk_size]
        batch_embeddings = generator.generate_embeddings(batch_texts)
        all_embeddings.append(batch_embeddings)
        
        # !!! KRITIKUS JAVÍTÁS: GPU memória felszabadítása minden darab után !!!
        generator._cleanup_memory()
            
    # Az összes darab embeddingjeinek összefűzése
    embeddings = np.concatenate(all_embeddings, axis=0)
    
    # Eredmények hozzáadása a DataFrame-hez
    if len(embeddings) == len(df):
        df['embedding'] = list(embeddings)
    else:
        logger.error(f"KRITIKUS HIBA: Az embeddingek száma ({len(embeddings)}) nem egyezik a DataFrame sorainak számával ({len(df)}). A program leáll.")
        return

    # Metaadatok generálása
    tqdm.pandas(desc="Metaadat JSON generálása")
    df['metadata_json'] = df.progress_apply(create_metadata_json, axis=1)

    # Kimeneti DataFrame és mentés Parquet formátumba
    final_df = df[['doc_id', 'text', 'embedding', 'metadata_json']]
    OUTPUT_PARQUET_PATH.parent.mkdir(parents=True, exist_ok=True)
    logger.info(f"Eredmények mentése a Parquet fájlba: {OUTPUT_PARQUET_PATH}")
    final_df.to_parquet(OUTPUT_PARQUET_PATH, index=False, compression='snappy')
    
    total_rows_processed = len(final_df)
    total_time_seconds = time.time() - start_time
    rows_per_second = total_rows_processed / total_time_seconds if total_time_seconds > 0 else 0
    
    # Összegzés
    print("\n" + "="*50)
    print("✅ FELDOLGOZÁS BEFEJEZVE")
    print(f"📄 Kimeneti fájl: {OUTPUT_PARQUET_PATH}")
    print(f"⏱️ Teljes idő: {total_time_seconds:.2f} másodperc ({total_time_seconds / 60:.2f} perc)")
    print(f"📊 Feldolgozott sorok: {total_rows_processed:,}")
    print(f"⚡ Átlagos sebesség: {rows_per_second:.2f} sor/mp")
    print("="*50)

# Fő folyamat futtatása
main()

In [ ]:
# === 5. VALIDÁCIÓ ===
logger.info("Kimeneti Parquet fájl validálása...")

if OUTPUT_PARQUET_PATH.exists():
    try:
        parquet_file = pq.ParquetFile(OUTPUT_PARQUET_PATH)
        file_num_rows = parquet_file.metadata.num_rows
        file_size_mb = OUTPUT_PARQUET_PATH.stat().st_size / (1024 * 1024)
        
        df_sample = pd.read_parquet(OUTPUT_PARQUET_PATH, engine='pyarrow', use_threads=True).head(5)
        sample_embedding = df_sample['embedding'].iloc[0]
        
        print("\n✅ VALIDÁCIÓ SIKERES!")
        print(f"  Fájl méret: {file_size_mb:.2f} MB")
        print(f"  Sorok száma: {file_num_rows:,}")
        print(f"  Oszlopok: {df_sample.columns.tolist()}")
        print(f"  Első embedding dimenziója: {len(sample_embedding)}")
        print("\n--- Minta Adatsor ---")
        display(df_sample)
        
    except Exception as e:
        logger.error(f"Hiba a Parquet fájl validálása közben: {e}")
        print(f"\n❌ HIBA a validáció során: {e}")
else:
    logger.error("A kimeneti Parquet fájl nem jött létre.")
    print("\n❌ HIBA: A kimeneti fájl nem található!")